<p align="center">
  <img src="https://www.dbs.ie/images/default-source/logos/dbs-logo-2019-small.png" />
</p>
This code is submitted by Yogeshwaran Shanmuganathan as a part of my final dissertation or thesis for the completion of "Master of Science in Data Analytics" at Dublin Business School, Dublin, Ireland.

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install pyspark

     |████████████████████████████████| 204.2MB 72kB/s 
     |████████████████████████████████| 204kB 41.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=1c789d8cf7ee9c9fb0403cbb98758800db75e1b6270c19c8770b3867366f3f76
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
# Load functionality to manipulate dataframes
from pyspark.sql import functions as fn
import matplotlib.pyplot as plt
from pyspark.sql.functions import stddev, mean, col
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
# Functionality for computing features
from pyspark.ml import feature, regression, classification, Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import feature, regression, classification, Pipeline
from pyspark.ml.feature import Tokenizer, VectorAssembler, HashingTF, Word2Vec, StringIndexer, OneHotEncoder
from pyspark.ml import clustering
from itertools import chain
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml import classification
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier, GBTClassifier
from pyspark.ml import evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
#Classification Report
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving master.csv to master.csv


In [ ]:
MAX_MEMORY = "45g"

spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [ ]:
# load master dataset
dfmaster = spark.read.format("csv").load("master.csv", delimiter = ",", header = True)


Preparations and Understanding before Modeling

In [ ]:
# create a 0/1 column for acquistions
dfmaster = dfmaster.\
  withColumn("labelacq", fn.when(col("status") == "acquired","1").otherwise("0"))

In [ ]:
# number of rows in master table
print(dfmaster.count())

49445


In [ ]:
dfmaster

DataFrame[_c0: string, permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: string, labelacq: string]

NAs and market column (with too many levels) handeling

In [ ]:
# check for missing values 
dfmaster.toPandas().isnull().sum()

_c0                           0
permalink                     1
name                          1
market                     3967
funding_total_usd             1
status                     1315
country_code               5273
city                       6116
funding_rounds                1
founded_year              10956
quarter_new               10956
age                       10956
permalink_agg                 1
count_investor                1
time_to_first_funding     24731
investor_country_codes    20954
funding_round_types       17110
total_raised_usd          49435
permalink_sub                 1
category_final            45459
perma                     20954
investor_country_code     20954
permaround                17110
funding_round_type        17110
labelacq                      0
dtype: int64

In [ ]:
# drop market columns because of too many level and better breakdown with the category_final column
dfmaster1 = dfmaster.drop("market")

In [ ]:
dfmaster1 = dfmaster1.toPandas()
dfmaster1

,_c0,permalink,name,funding_total_usd,status,country_code,city,funding_rounds,founded_year,quarter_new,age,permalink_agg,count_investor,time_to_first_funding,investor_country_codes,funding_round_types,total_raised_usd,permalink_sub,category_final,perma,investor_country_code,permaround,funding_round_type,labelacq
0,0,/organization/1lay,1Lay,"1,70,000",operating,None,None,1,2013,Q3,1.5,/organization/1lay,0.0,None,None,None,None,/organization/1lay,None,None,None,None,None,0
1,1,/organization/24pagebooks,24PageBooks,"50,000",closed,USA,Rochester,1,2010,Q1,5.0,/organization/24pagebooks,1.0,1.25,FRA,seed,None,/organization/24pagebooks,None,/organization/24pagebooks,FRA,/organization/24pagebooks,seed,0
2,2,/organization/5min,5min Media,"1,28,00,000",acquired,USA,New York,3,2007,Q1,8.0,/organization/5min,3.0,0.75,"USA, USA, USA","venture, venture, venture",None,/organization/5min,None,/organization/5min,USA,/organization/5min,venture,1
3,3,/organization/abpathfinder,ABPathfinder,"9,60,000",operating,USA,Overland Park,3,2010,Q2,4.75,/organization/abpathfinder,4.0,2.5,"USA, USA, USA","venture, venture, venture, venture",None,/organization/abpathfinder,None,/organization/abpathfinder,USA,/organization/abpathfinder,venture,0
4,4,/organization/acid-labs,Acid Labs,-,operating,USA,Santa Monica,1,2003,Q1,12.0,/organization/acid-labs,0.0,None,None,None,None,/organization/acid-labs,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49440,49440,/organization/yieldmo,Yieldmo,"2,21,00,000",operating,USA,New York,3,2012,Q2,2.75,/organization/yieldmo,12.0,0.25,"USA, USA, USA, USA, USA, ISR, USA, USA, USA, U...","venture, venture, venture, venture, venture, v...",None,/organization/yieldmo,None,/organization/yieldmo,USA,/organization/yieldmo,venture,0
49441,49441,/organization/youxinpai,Youxinpai,"3,00,00,000",operating,CHN,Beijing,1,2011,Q3,3.5,/organization/youxinpai,4.0,1.75,"CHN, USA, CHN","venture, venture, venture, venture",None,/organization/youxinpai,None,/organization/youxinpai,CHN,/organization/youxinpai,venture,0
49442,49442,/organization/zeropercent-us,ZeroPercent.us,-,operating,USA,Chicago,1,2011,Q3,3.5,/organization/zeropercent-us,1.0,2.5,USA,seed,None,/organization/zeropercent-us,None,/organization/zeropercent-us,USA,/organization/zeropercent-us,seed,0
49443,49443,/organization/zinch,Zinch,"54,86,842",acquired,USA,Orem,3,2007,Q1,8.0,/organization/zinch,3.0,0.75,"USA, USA","venture, venture, angel",None,/organization/zinch,None,/organization/zinch,USA,/organization/zinch,venture,1


In [ ]:
# Replace NaN with mode for categorical variables
dfmaster1['total_raised_usd'] = dfmaster1['total_raised_usd'].fillna(dfmaster1['total_raised_usd'].mode()[0])
dfmaster1['time_to_first_funding'] = dfmaster1['time_to_first_funding'].fillna(dfmaster1['time_to_first_funding'].mode()[0])
dfmaster1['founded_year'] = dfmaster1['founded_year'].fillna(dfmaster1['founded_year'].mode()[0])
dfmaster1['age'] = dfmaster1['age'].fillna(dfmaster1['age'].mode()[0])
dfmaster1['status'] = dfmaster1['status'].fillna(dfmaster1['status'].mode()[0])
dfmaster1['country_code'] = dfmaster1['country_code'].fillna(dfmaster1['country_code'].mode()[0])
dfmaster1['city'] = dfmaster1['city'].fillna(dfmaster1['city'].mode()[0])
dfmaster1['quarter_new'] = dfmaster1['quarter_new'].fillna(dfmaster1['quarter_new'].mode()[0])
dfmaster1['investor_country_codes'] = dfmaster1['investor_country_codes'].fillna(dfmaster1['investor_country_codes'].mode()[0])
dfmaster1['funding_round_types'] = dfmaster1['funding_round_types'].fillna(dfmaster1['funding_round_types'].mode()[0])
dfmaster1['permaround'] = dfmaster1['permaround'].fillna(dfmaster1['permaround'].mode()[0])
dfmaster1['investor_country_code'] = dfmaster1['investor_country_code'].fillna(dfmaster1['investor_country_code'].mode()[0])
dfmaster1['funding_round_type'] = dfmaster1['funding_round_type'].fillna(dfmaster1['funding_round_type'].mode()[0])
dfmaster1['category_final'] = dfmaster1['category_final'].fillna(dfmaster1['category_final'].mode()[0])
dfmaster1['perma'] = dfmaster1['perma'].fillna(dfmaster1['perma'].mode()[0])

In [ ]:
# check for missing values 
dfmaster1.isnull().sum()

_c0                       0
permalink                 1
name                      1
funding_total_usd         1
status                    0
country_code              0
city                      0
funding_rounds            1
founded_year              0
quarter_new               0
age                       0
permalink_agg             1
count_investor            1
time_to_first_funding     0
investor_country_codes    0
funding_round_types       0
total_raised_usd          0
permalink_sub             1
category_final            0
perma                     0
investor_country_code     0
permaround                0
funding_round_type        0
labelacq                  0
dtype: int64

In [ ]:
# drop rows with missing values
dfmaster1drop = dfmaster1.dropna()

In [ ]:
print(dfmaster1drop.count())

_c0                       49444
permalink                 49444
name                      49444
funding_total_usd         49444
status                    49444
country_code              49444
city                      49444
funding_rounds            49444
founded_year              49444
quarter_new               49444
age                       49444
permalink_agg             49444
count_investor            49444
time_to_first_funding     49444
investor_country_codes    49444
funding_round_types       49444
total_raised_usd          49444
permalink_sub             49444
category_final            49444
perma                     49444
investor_country_code     49444
permaround                49444
funding_round_type        49444
labelacq                  49444
dtype: int64


In [ ]:
sql = SQLContext(spark)

In [ ]:
dfmaster2 = sql.createDataFrame(dfmaster1drop)

In [ ]:
display(dfmaster2)

DataFrame[_c0: string, permalink: string, name: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: string, labelacq: string]

String indexer, one hot encoder and casting to numerics

In [ ]:
# create index for categorical variables
# use pipline to apply indexer
list1 = ["country_code","city","quarter_new","investor_country_code","funding_round_type","category_final"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dfmaster2) for column in list1]
pipelineindex = Pipeline(stages=indexers).fit(dfmaster2)
dfmasternew = pipelineindex.transform(dfmaster2)

In [ ]:
# convert string to double for numerical variables
dfmasternew = dfmasternew.\
  withColumn("numeric_funding_rounds", dfmasternew["funding_rounds"].cast("int")).\
  withColumn("numeric_age", dfmasternew["age"].cast("int")).\
  withColumn("numeric_count_investor", dfmasternew["count_investor"].cast("int")).\
  withColumn("numeric_time_to_first_funding", dfmasternew["time_to_first_funding"].cast("int")).\
  withColumn("numeric_total_raised_usd", dfmasternew["total_raised_usd"].cast("int")).\
  withColumn("label", dfmasternew["labelacq"].cast("int"))

In [ ]:
dfmasternew = dfmasternew.\
  withColumn("funding_round_type", dfmasternew["funding_round_type"].cast("double")).\
  withColumn("country_code_index", dfmasternew["country_code_index"].cast("double")).\
  withColumn("city_index", dfmasternew["city_index"].cast("double")).\
  withColumn("quarter_new_index", dfmasternew["quarter_new_index"].cast("double")).\
  withColumn("labelacq", dfmasternew["labelacq"].cast("double"))

In [ ]:
# save
dfone = dfmasternew

In [ ]:
display(dfone)

DataFrame[_c0: string, permalink: string, name: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: double, labelacq: double, country_code_index: double, city_index: double, quarter_new_index: double, investor_country_code_index: double, funding_round_type_index: double, category_final_index: double, numeric_funding_rounds: int, numeric_age: int, numeric_count_investor: int, numeric_time_to_first_funding: int, numeric_total_raised_usd: int, label: int]

In [ ]:
print(dfone.count())

49444


In [ ]:
# list of index columns of categorical variables for the onehotencoder
list2 = dfone.columns[24:30]
list2

['country_code_index',
 'city_index',
 'quarter_new_index',
 'investor_country_code_index',
 'funding_round_type_index',
 'category_final_index']

In [ ]:
# create sparse matrix of indexed categorical columns
# use pipline to apply the encoder
onehotencoder_stages = [OneHotEncoder(inputCol=c, outputCol='onehotencoded_' + c) for c in list2]
pipelineonehot = Pipeline(stages=onehotencoder_stages)
pipeline_mode = pipelineonehot.fit(dfone)
df_coded = pipeline_mode.transform(dfone)

In [ ]:
df_coded.show()

+---+--------------------+--------------------+-----------------+---------+------------+-------------+--------------+------------+-----------+----+--------------------+--------------+---------------------+----------------------+--------------------+----------------+--------------------+--------------+--------------------+---------------------+--------------------+------------------+--------+------------------+----------+-----------------+---------------------------+------------------------+--------------------+----------------------+-----------+----------------------+-----------------------------+------------------------+-----+--------------------------------+------------------------+-------------------------------+-----------------------------------------+--------------------------------------+----------------------------------+
|_c0|           permalink|                name|funding_total_usd|   status|country_code|         city|funding_rounds|founded_year|quarter_new| age|       perm


Data split, defining vector assemblers & standard scaler and creating labellist

In [ ]:
# split dataset into training, validaiton and testing dataset
training_df, validation_df, testing_df = df_coded.randomSplit([0.6, 0.3, 0.1])

In [ ]:
training_df.columns[30:35]

['numeric_funding_rounds',
 'numeric_age',
 'numeric_count_investor',
 'numeric_time_to_first_funding',
 'numeric_total_raised_usd']

In [ ]:
training_df.columns[36:42]

['onehotencoded_country_code_index',
 'onehotencoded_city_index',
 'onehotencoded_quarter_new_index',
 'onehotencoded_investor_country_code_index',
 'onehotencoded_funding_round_type_index',
 'onehotencoded_category_final_index']

In [ ]:
# define vector assembler with the features for the modelling
vanum = VectorAssembler(). \
      setInputCols(training_df.columns[30:35]). \
      setOutputCol('features_nonstd')

In [ ]:
# define vector assembler with the features for the modelling
vacate = VectorAssembler(). \
      setInputCols(training_df.columns[36:42]). \
      setOutputCol('featurescate')

In [ ]:
va = VectorAssembler(). \
      setInputCols(['featuresnum','featurescate']). \
      setOutputCol('features')

In [ ]:
std = feature.StandardScaler(withMean=True, withStd=True).setInputCol('features_nonstd').setOutputCol('featuresnum')

In [ ]:
# suffix for investor country code because intersection with county_code of  the companies
invcc = ['{}_{}'.format(a, "investor") for a in indexers[3].labels]

In [ ]:
# define labellist by using the indexer stages for displaying the weights & loadings
labellist = training_df.columns[30:35] + indexers[0].labels + indexers[1].labels + indexers[2].labels + invcc + indexers[4].labels + indexers[5].labels

In [ ]:
# null dummy for onehotencoded_country_code_index
print("null dummy for onehotencoded_country_code_index")
print(len(indexers[0].labels))
print(indexers[0].labels)
# null dummy for onehotencoded_city_index
print("null dummy for onehotencoded_city_index")
print(indexers[1].labels)
print(len(indexers[1].labels))
# null dummy for onehotencoded_quarter_new_index
print("null dummy for onehotencoded_quarter_new_index")
print(len(indexers[2].labels))
print(indexers[2].labels)
# null dummy for onehotencoded_investor_country_code_index
print("null dummy for onehotencoded_investor_country_code_index")
print(len(invcc))
print(invcc)
# null dummy for onehotencoded_funding_round_type_index
print("null dummy for onehotencoded_funding_round_type_index")
print(len(indexers[4].labels))
print(indexers[4].labels)
# null dummy for onehotencoded_category_final_index
print("null dummy for onehotencoded_category_final_index")
print(len(indexers[5].labels))
print(indexers[5].labels)

null dummy for onehotencoded_country_code_index
115
['USA', 'GBR', 'CAN', 'CHN', 'DEU', 'FRA', 'IND', 'ISR', 'ESP', 'RUS', 'SWE', 'AUS', 'ITA', 'NLD', 'IRL', 'SGP', 'BRA', 'CHL', 'JPN', 'KOR', 'CHE', 'DNK', 'FIN', 'ARG', 'BEL', 'HKG', 'TUR', 'AUT', 'NOR', 'POL', 'MEX', 'PRT', 'BGR', 'ARE', 'NZL', 'IDN', 'ZAF', 'CZE', 'MYS', 'UKR', 'EST', 'HUN', 'TWN', 'THA', 'COL', 'PHL', 'GRC', 'LTU', 'PER', 'NGA', 'KEN', 'EGY', 'LUX', 'ROM', 'VNM', 'DZA', 'JOR', 'PAK', 'ISL', 'SVK', 'LBN', 'CYP', 'LVA', 'URY', 'CYM', 'GHA', 'SVN', 'KHM', 'SRB', 'UGA', 'HRV', 'BGD', 'SAU', 'TZA', 'CRI', 'PAN', 'BMU', 'BWA', 'GTM', 'AZE', 'BHR', 'BLR', 'DOM', 'MAR', 'MLT', 'SLV', 'TUN', 'ARM', 'BHS', 'CMR', 'ECU', 'GIB', 'KWT', 'LAO', 'MDA', 'MKD', 'MMR', 'NIC', 'NPL', 'ALB', 'BRN', 'CIV', 'JAM', 'JEY', 'LIE', 'MAF', 'MCO', 'MOZ', 'MUS', 'OMN', 'SOM', 'SYC', 'TTO', 'UZB', 'ZWE']
null dummy for onehotencoded_city_index
['San Francisco', 'New York', 'London', 'Palo Alto', 'Austin', 'Cambridge', 'Seattle', 'Chicago', 'Los

# MODELLING

## Gradient Boosted Trees

In [ ]:
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=250)

In [ ]:
gbt_pipeline = Pipeline(stages=[vanum, std, vacate, va, gbt]).fit(training_df)

In [ ]:
dfgbt = gbt_pipeline.transform(validation_df)

In [ ]:
# define multiclass classification evaluator
mce = MulticlassClassificationEvaluator()

In [ ]:
# print the areas under the curve for the gradient boosted trees model pipeline
print("Gradient Boosted Trees: AUC = {}".format(mce.evaluate(dfgbt)))

Gradient Boosted Trees: AUC = 0.8938379312203558


In [ ]:
# print the accuracies for the gradient boosted trees model pipeline
print(dfgbt.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Gradient Boosted Tree")).show())

+----------------------------------+
|Accuracy for Gradient Boosted Tree|
+----------------------------------+
|                0.9221085229567794|
+----------------------------------+

None


# Neural Networks

### Standard Model

In [ ]:
# define neural networks (MultilayerPerceptron) classifier
mlp = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setLayers([4399, 2]).\
    setFeaturesCol('features')

In [ ]:
# define and fit neural network pipeline 
mlp_simple_model = Pipeline(stages=[vanum, std, vacate, va, mlp]).fit(training_df)

In [ ]:
# define evaluators for accuracy and area under the curve
evaluator = MulticlassClassificationEvaluator().setMetricName("accuracy")
evaluatorauc = MulticlassClassificationEvaluator()

In [ ]:
# apply pipeline to validation dataset
dfnn = mlp_simple_model.transform(validation_df)

In [ ]:
# print accuracy and area under the curve for NN
print(evaluator.evaluate(dfnn))
print(evaluatorauc.evaluate(dfnn))

0.919482967550828
0.8874716677080274


### Models with Hidden Layers

In [ ]:
# define neural networks (MultilayerPerceptron) classifier with hidden layers
mlp2 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30,30, 2])
mlp3 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,10,10, 2])
mlp4 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,20,20, 2])
mlp5 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30, 2])
mlp6 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30,30,30, 2])

In [ ]:
# define and fit pipeline
mlp2_model = Pipeline(stages=[vanum, std, vacate, va, mlp2]).fit(training_df)
mlp3_model = Pipeline(stages=[vanum, std, vacate, va, mlp3]).fit(training_df)
mlp4_model = Pipeline(stages=[vanum, std, vacate, va, mlp4]).fit(training_df)
mlp5_model = Pipeline(stages=[vanum, std, vacate, va, mlp5]).fit(training_df)
mlp6_model = Pipeline(stages=[vanum, std, vacate, va, mlp6]).fit(training_df)

In [ ]:
# apply and fit model to validation dataset
dfnn2 = mlp2_model.transform(validation_df)
dfnn3 = mlp3_model.transform(validation_df)
dfnn4 = mlp4_model.transform(validation_df)
dfnn5 = mlp5_model.transform(validation_df)
dfnn6 = mlp6_model.transform(validation_df)

## Performance

In [ ]:
# print accuracy and area under the curve
print("NN Model with 2 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn2)))
print("NN Model with 2 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn2)))
print("____________________________________________________________________________")
print("NN Model with 2 hidden layers and 10 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn3)))
print("NN Model with 2 hidden layers and 10 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn3)))
print("____________________________________________________________________________")
print("NN Model with 2 hidden layers and 20 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn4)))
print("NN Model with 2 hidden layers and 20 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn4)))
print("____________________________________________________________________________")
print("NN Model with 1 hidden layer and 30 neurons: Accuracy = {}".format(evaluator.evaluate(dfnn5)))
print("NN Model with 1 hidden layer and 30 neurons: AUC = {}".format(evaluatorauc.evaluate(dfnn5)))
print("____________________________________________________________________________")
print("NN Model with 3 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn6)))
print("NN Model with 3 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn6)))

NN Model with 2 hidden layers and 30 neurons each: Accuracy = 0.9174633095462502
NN Model with 2 hidden layers and 30 neurons each: AUC = 0.8950379299317124
____________________________________________________________________________
NN Model with 2 hidden layers and 10 neurons each: Accuracy = 0.9145011444728692
NN Model with 2 hidden layers and 10 neurons each: AUC = 0.8953964473748464
____________________________________________________________________________
NN Model with 2 hidden layers and 20 neurons each: Accuracy = 0.9164534805439613
NN Model with 2 hidden layers and 20 neurons each: AUC = 0.8968682586838205
____________________________________________________________________________
NN Model with 1 hidden layer and 30 neurons: Accuracy = 0.9120775548673757
NN Model with 1 hidden layer and 30 neurons: AUC = 0.8950377241505765
____________________________________________________________________________
NN Model with 3 hidden layers and 30 neurons each: Accuracy = 0.921704591355

# TESTING PERFORMANCE

We tested with the best performing (AUC and Accuracy) Gradient Bossted Trees and Neural Network Model.

In [ ]:
# Gradient Boosted Trees Classifier
dfgbt_test = gbt_pipeline.transform(testing_df)
print("Gradient Boosted Trees: AUC = {}".format(mce.evaluate(dfgbt_test)))
print(dfgbt_test.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Gradient Boosted Trees")).show())

Gradient Boosted Trees: AUC = 0.8907509053766469
+-----------------------------------+
|Accuracy for Gradient Boosted Trees|
+-----------------------------------+
|                 0.9196754563894524|
+-----------------------------------+

None


In [ ]:
gbt_true = dfgbt_test.select(['label']).collect()
gbt_pred = dfgbt_test.select(['prediction']).collect()
print(classification_report(gbt_true, gbt_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4542
           1       0.43      0.06      0.10       388

    accuracy                           0.92      4930
   macro avg       0.68      0.53      0.53      4930
weighted avg       0.89      0.92      0.89      4930



In [ ]:
# Best performing neural network model
dfnntest = mlp6_model.transform(testing_df)
print("Neural Network Model with 3 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnntest)))
print("Neural Network Model with 3 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnntest)))

Neural Network Model with 3 hidden layers and 30 neurons each: Accuracy = 0.9186612576064909
Neural Network Model with 3 hidden layers and 30 neurons each: AUC = 0.8939215389008038


In [ ]:
nn_true = dfnntest.select(['label']).collect()
nn_pred = dfnntest.select(['prediction']).collect()
print(classification_report(nn_true, nn_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4542
           1       0.42      0.09      0.15       388

    accuracy                           0.92      4930
   macro avg       0.68      0.54      0.55      4930
weighted avg       0.89      0.92      0.89      4930

